In [1]:

import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sb

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import f_regression, VarianceThreshold
from sklearn.metrics import mean_absolute_error, r2_score

from keras.layers import Normalization, Dense, Dropout
from keras.optimizers import Adam
from keras.models import Sequential
from keras import regularizers

from keras.callbacks import ModelCheckpoint
from keras import backend as K


## Import Data from .csv File

In [2]:

df = pd.read_csv(r'365finalproject.csv', encoding= 'unicode_escape')


In [3]:
df

,PlayerID,Player,Age,Games,Minutes,FGM,FGA,3PM,3PA,FTM,...,PF,PTS,FG%,3P%,FT%,MPG,PPG,RPG,APG,WS_25
0,duartch01,Chris Duarte,24,55,1541,268,621,94,255,90,...,95,720,0.432,0.369,0.804,28.0,13.1,4.1,2.1,0.0
1,marshna01,Naji Marshall,23,32,700,82,209,29,83,53,...,58,246,0.392,0.349,0.707,21.9,7.7,4.6,2.8,2.5
2,pritcpa01,Payton Pritchard,23,66,1268,184,418,102,248,40,...,104,510,0.440,0.411,0.889,19.2,7.7,2.4,1.8,0.6
3,windldy01,Dylan Windler,24,31,513,57,130,26,77,21,...,37,161,0.438,0.338,0.778,16.5,5.2,3.5,1.1,0.8
4,clarkbr01,Brandon Clarke,23,58,1300,296,479,23,64,85,...,100,700,0.618,0.359,0.759,22.4,12.1,5.9,1.4,6.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,kunneke01,Kevin Kunnert,22,64,701,105,215,0,0,21,...,151,231,0.488,0.000,0.636,11.0,3.6,3.4,0.7,4.1
1032,mcneila01,Larry McNeill,23,54,516,106,220,0,0,99,...,76,311,0.482,0.000,0.707,9.6,5.8,2.7,0.4,4.3
1033,ratleed01,Ed Ratleff,23,81,1773,254,585,0,0,103,...,182,611,0.434,0.000,0.798,21.9,7.5,3.5,2.2,4.5
1034,wattssl01,Slick Watts,22,62,1424,198,510,0,0,100,...,207,496,0.388,0.000,0.645,23.0,8.0,2.9,5.7,3.3


In [4]:
## remove 3 pt statistics
df = df.drop(df.columns[[7, 8, 19]], axis=1)

In [5]:
df

,PlayerID,Player,Age,Games,Minutes,FGM,FGA,FTM,FTA,ORB,...,BLK,PF,PTS,FG%,FT%,MPG,PPG,RPG,APG,WS_25
0,duartch01,Chris Duarte,24,55,1541,268,621,90,112,41,...,10,95,720,0.432,0.804,28.0,13.1,4.1,2.1,0.0
1,marshna01,Naji Marshall,23,32,700,82,209,53,75,15,...,10,58,246,0.392,0.707,21.9,7.7,4.6,2.8,2.5
2,pritcpa01,Payton Pritchard,23,66,1268,184,418,40,45,33,...,9,104,510,0.440,0.889,19.2,7.7,2.4,1.8,0.6
3,windldy01,Dylan Windler,24,31,513,57,130,21,27,20,...,12,37,161,0.438,0.778,16.5,5.2,3.5,1.1,0.8
4,clarkbr01,Brandon Clarke,23,58,1300,296,479,85,112,92,...,48,100,700,0.618,0.759,22.4,12.1,5.9,1.4,6.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,kunneke01,Kevin Kunnert,22,64,701,105,215,21,33,83,...,54,151,231,0.488,0.636,11.0,3.6,3.4,0.7,4.1
1032,mcneila01,Larry McNeill,23,54,516,106,220,99,140,60,...,6,76,311,0.482,0.707,9.6,5.8,2.7,0.4,4.3
1033,ratleed01,Ed Ratleff,23,81,1773,254,585,103,129,93,...,27,182,611,0.434,0.798,21.9,7.5,3.5,2.2,4.5
1034,wattssl01,Slick Watts,22,62,1424,198,510,100,155,72,...,13,207,496,0.388,0.645,23.0,8.0,2.9,5.7,3.3


In [6]:

names_ds = df[["PlayerID", "Player"]]
features_ds_raw = df.loc[:,'Age':'APG']
labels_ds = df[["WS_25"]]


In [7]:
names_ds

,PlayerID,Player
0,duartch01,Chris Duarte
1,marshna01,Naji Marshall
2,pritcpa01,Payton Pritchard
3,windldy01,Dylan Windler
4,clarkbr01,Brandon Clarke
...,...,...
1031,kunneke01,Kevin Kunnert
1032,mcneila01,Larry McNeill
1033,ratleed01,Ed Ratleff
1034,wattssl01,Slick Watts


In [8]:
features_ds_raw

,Age,Games,Minutes,FGM,FGA,FTM,FTA,ORB,TRB,AST,STL,BLK,PF,PTS,FG%,FT%,MPG,PPG,RPG,APG
0,24,55,1541,268,621,90,112,41,226,114,56,10,95,720,0.432,0.804,28.0,13.1,4.1,2.1
1,23,32,700,82,209,53,75,15,148,88,26,10,58,246,0.392,0.707,21.9,7.7,4.6,2.8
2,23,66,1268,184,418,40,45,33,158,120,37,9,104,510,0.440,0.889,19.2,7.7,2.4,1.8
3,24,31,513,57,130,21,27,20,107,33,19,12,37,161,0.438,0.778,16.5,5.2,3.5,1.1
4,23,58,1300,296,479,85,112,92,345,81,32,48,100,700,0.618,0.759,22.4,12.1,5.9,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1031,22,64,701,105,215,21,33,83,217,43,10,54,151,231,0.488,0.636,11.0,3.6,3.4,0.7
1032,23,54,516,106,220,99,140,60,146,24,35,6,76,311,0.482,0.707,9.6,5.8,2.7,0.4
1033,23,81,1773,254,585,103,129,93,286,181,90,27,182,611,0.434,0.798,21.9,7.5,3.5,2.2
1034,22,62,1424,198,510,100,155,72,182,351,115,13,207,496,0.388,0.645,23.0,8.0,2.9,5.7


In [9]:
labels_ds

,WS_25
0,0.0
1,2.5
2,0.6
3,0.8
4,6.3
...,...
1031,4.1
1032,4.3
1033,4.5
1034,3.3


## Normalization of Data

In [10]:

normalization_layer = tf.keras.layers.Normalization(axis=-1)
normalization_layer.adapt(features_ds_raw)


In [11]:

features_ds_normalized = normalization_layer(features_ds_raw)


In [12]:
features_ds = np.array(features_ds_normalized)

In [13]:
features_ds

array([[ 1.587192  , -1.133756  ,  0.03314409, ...,  0.9669427 ,
         0.09333976,  0.11713215],
       [ 0.8004314 , -2.9718945 , -1.2081058 , ..., -0.21646269,
         0.3200158 ,  0.55341506],
       [ 0.8004314 , -0.2546463 , -0.36978245, ..., -0.21646269,
        -0.6773586 , -0.06984619],
       ...,
       [ 0.8004314 ,  0.94413966,  0.37555787, ..., -0.26029247,
        -0.17867143,  0.17945836],
       [ 0.01367071, -0.5743225 , -0.13953872, ..., -0.1507179 ,
        -0.4506826 ,  2.3608725 ],
       [ 0.8004314 , -0.33456534, -0.44653037, ..., -0.23837757,
         1.0000439 , -0.81775963]], dtype=float32)

## K Fold Validation with 20Folds

In [63]:
from sklearn.model_selection import KFold

In [112]:
kf = KFold(n_splits=10)

In [146]:
r2_list = []

In [147]:
best_r2 = -np.inf
best_weights = None

In [148]:
y = labels_ds

In [149]:
X = features_ds

In [150]:
for train_index, test_index in kf.split(X):
    x_train, x_test = X[train_index], X[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    model = Sequential()
    model.add(Dense(units=1024, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.25))
    model.add(Dense(units=512, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.25))
    model.add(Dense(units=256, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.25))
    model.add(Dense(units=128, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
    model.add(Dropout(0.25))
    
    model.add(Dense(units=1, activation='linear'))
    
    
    callback = tf.keras.callbacks.EarlyStopping (
        monitor = 'mean_absolute_error',
        patience = 5,
        restore_best_weights = True
    )

    model.compile(loss='mse', optimizer=Adam(learning_rate=0.001), metrics=['mean_absolute_error'])
    model.fit(x_train, y_train, epochs=50, callbacks = [callback], validation_data=(x_test, y_test))
    
    y_pred = model.predict(x_test)
    mae = mean_absolute_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    
    if r2 > best_r2:
        best_r2 = r2
        best_model = model
    
    mae_list.append(mae)
    r2_list.append(r2)

Epoch 1/50
30/30 [==============================] - 2s 27ms/step - loss: 24.3037 - mean_absolute_error: 2.7304 - val_loss: 15.7574 - val_mean_absolute_error: 1.8052
Epoch 2/50
30/30 [==============================] - 0s 17ms/step - loss: 18.6485 - mean_absolute_error: 2.3204 - val_loss: 15.0561 - val_mean_absolute_error: 1.9538
Epoch 3/50
30/30 [==============================] - 1s 18ms/step - loss: 16.9147 - mean_absolute_error: 2.2992 - val_loss: 12.6245 - val_mean_absolute_error: 1.7522
Epoch 4/50
30/30 [==============================] - 1s 19ms/step - loss: 15.5590 - mean_absolute_error: 2.2828 - val_loss: 11.5278 - val_mean_absolute_error: 1.7582
Epoch 5/50
30/30 [==============================] - 1s 18ms/step - loss: 14.3970 - mean_absolute_error: 2.2252 - val_loss: 10.4771 - val_mean_absolute_error: 1.6998
Epoch 6/50
30/30 [==============================] - 1s 20ms/step - loss: 14.1174 - mean_absolute_error: 2.2692 - val_loss: 11.9831 - val_mean_absolute_error: 2.1006
Epoch 7/50

30/30 [==============================] - 2s 27ms/step - loss: 23.4880 - mean_absolute_error: 2.5900 - val_loss: 20.1178 - val_mean_absolute_error: 2.4342
Epoch 2/50
30/30 [==============================] - 1s 17ms/step - loss: 19.0825 - mean_absolute_error: 2.3621 - val_loss: 16.8190 - val_mean_absolute_error: 2.2936
Epoch 3/50
30/30 [==============================] - 1s 19ms/step - loss: 16.2227 - mean_absolute_error: 2.2299 - val_loss: 16.2978 - val_mean_absolute_error: 2.3247
Epoch 4/50
30/30 [==============================] - 0s 17ms/step - loss: 15.2194 - mean_absolute_error: 2.2178 - val_loss: 14.2296 - val_mean_absolute_error: 2.3401
Epoch 5/50
30/30 [==============================] - 1s 20ms/step - loss: 14.0335 - mean_absolute_error: 2.2112 - val_loss: 13.5393 - val_mean_absolute_error: 2.2682
Epoch 6/50
30/30 [==============================] - 1s 19ms/step - loss: 13.1369 - mean_absolute_error: 2.1922 - val_loss: 12.8753 - val_mean_absolute_error: 2.2932
Epoch 7/50
30/30 [===

4/4 [==============================] - 0s 4ms/step
Epoch 1/50
30/30 [==============================] - 2s 28ms/step - loss: 23.0986 - mean_absolute_error: 2.5721 - val_loss: 20.4696 - val_mean_absolute_error: 2.3994
Epoch 2/50
30/30 [==============================] - 0s 16ms/step - loss: 18.5833 - mean_absolute_error: 2.3143 - val_loss: 17.4982 - val_mean_absolute_error: 2.2991
Epoch 3/50
30/30 [==============================] - 1s 19ms/step - loss: 16.6573 - mean_absolute_error: 2.2122 - val_loss: 18.1340 - val_mean_absolute_error: 2.5111
Epoch 4/50
30/30 [==============================] - 1s 19ms/step - loss: 15.0903 - mean_absolute_error: 2.2065 - val_loss: 17.1550 - val_mean_absolute_error: 2.4844
Epoch 5/50
30/30 [==============================] - 1s 19ms/step - loss: 14.1267 - mean_absolute_error: 2.1955 - val_loss: 13.2216 - val_mean_absolute_error: 2.1723
Epoch 6/50
30/30 [==============================] - 1s 19ms/step - loss: 13.1270 - mean_absolute_error: 2.1367 - val_loss: 1

30/30 [==============================] - 1s 17ms/step - loss: 7.8460 - mean_absolute_error: 1.9557 - val_loss: 10.6174 - val_mean_absolute_error: 2.2086
Epoch 28/50
4/4 [==============================] - 0s 4ms/step
Epoch 1/50
30/30 [==============================] - 2s 26ms/step - loss: 23.8982 - mean_absolute_error: 2.6509 - val_loss: 17.2991 - val_mean_absolute_error: 1.9787
Epoch 2/50
30/30 [==============================] - 0s 16ms/step - loss: 19.0598 - mean_absolute_error: 2.3753 - val_loss: 15.3918 - val_mean_absolute_error: 1.9224
Epoch 3/50
30/30 [==============================] - 0s 16ms/step - loss: 16.4025 - mean_absolute_error: 2.2371 - val_loss: 14.7941 - val_mean_absolute_error: 2.0978
Epoch 4/50
30/30 [==============================] - 1s 18ms/step - loss: 15.4767 - mean_absolute_error: 2.2790 - val_loss: 12.0559 - val_mean_absolute_error: 1.8210
Epoch 5/50
30/30 [==============================] - 0s 16ms/step - loss: 13.9088 - mean_absolute_error: 2.1649 - val_loss: 1

Epoch 11/50
30/30 [==============================] - 0s 15ms/step - loss: 10.5541 - mean_absolute_error: 2.1228 - val_loss: 11.3701 - val_mean_absolute_error: 2.2156
Epoch 12/50
30/30 [==============================] - 1s 18ms/step - loss: 10.1081 - mean_absolute_error: 2.0979 - val_loss: 11.5616 - val_mean_absolute_error: 2.1924
Epoch 13/50
30/30 [==============================] - 0s 16ms/step - loss: 9.7792 - mean_absolute_error: 2.0723 - val_loss: 10.2913 - val_mean_absolute_error: 2.1215
Epoch 14/50
4/4 [==============================] - 0s 4ms/step
Epoch 1/50
30/30 [==============================] - 2s 26ms/step - loss: 22.9860 - mean_absolute_error: 2.5456 - val_loss: 19.2000 - val_mean_absolute_error: 2.2061
Epoch 2/50
30/30 [==============================] - 0s 16ms/step - loss: 17.9861 - mean_absolute_error: 2.2498 - val_loss: 17.0331 - val_mean_absolute_error: 2.1933
Epoch 3/50
30/30 [==============================] - 1s 17ms/step - loss: 16.5697 - mean_absolute_error: 2.2539

30/30 [==============================] - 0s 15ms/step - loss: 7.1050 - mean_absolute_error: 1.8662 - val_loss: 9.3682 - val_mean_absolute_error: 2.2070
Epoch 47/50
30/30 [==============================] - 0s 16ms/step - loss: 6.5759 - mean_absolute_error: 1.7901 - val_loss: 9.8413 - val_mean_absolute_error: 2.2667
Epoch 48/50
30/30 [==============================] - 1s 17ms/step - loss: 6.5246 - mean_absolute_error: 1.7687 - val_loss: 8.8397 - val_mean_absolute_error: 2.1966
Epoch 49/50
30/30 [==============================] - 1s 17ms/step - loss: 7.0482 - mean_absolute_error: 1.8710 - val_loss: 10.0668 - val_mean_absolute_error: 2.2745
Epoch 50/50
4/4 [==============================] - 0s 4ms/step
Epoch 1/50
30/30 [==============================] - 3s 27ms/step - loss: 21.6900 - mean_absolute_error: 2.4402 - val_loss: 17.6100 - val_mean_absolute_error: 2.1721
Epoch 2/50
30/30 [==============================] - 1s 17ms/step - loss: 17.7356 - mean_absolute_error: 2.2142 - val_loss: 16.0

In [151]:
print("done")

done


In [152]:
mean_mae = np.mean(mae_list)
print("Mean MAE:", mean_mae)
mean_r2 = np.mean(r2_list)
print("Mean R2:", mean_r2)

Mean MAE: 2.214293824821388
Mean R2: 0.22676953120263468


In [153]:
best_r2 = np.max(r2_list)
best_mae = np.min(mae_list)
print("Best MAE:", best_mae)
print("Best R2:", best_r2)

Best MAE: 1.8703169685143688
Best R2: 0.4099043608632037


In [154]:
best_model.save_weights('best_weights5.h5')

In [155]:
n_train1, n_test1, x_train1, x_test1, y_train1, y_test1 = train_test_split(names_ds, X, y, test_size=0.3)


In [156]:
model.load_weights('best_weights5.h5') ## 5th run was the best

### R2 on entire dataset

In [157]:
model.evaluate(X, y)

33/33 [==============================] - 0s 6ms/step - loss: 8.3739 - mean_absolute_error: 1.9169


[8.373924255371094, 1.9169092178344727]

In [158]:
y_pred1 = model.predict(X)

33/33 [==============================] - 0s 5ms/step


In [159]:
r2_score1 = r2_score(y, y_pred1)
print(r2_score1)

0.4475711739233199


## Split Data into Training and Testing Datasets

In [ ]:

##n_train, n_test, x_train, x_test, y_train, y_test = train_test_split(names_ds, features_ds, labels_ds, test_size=0.3)


In [ ]:

##n_train


In [ ]:
##x_train

In [ ]:
##y_train

In [ ]:
##n_test

In [ ]:
##x_test

In [ ]:
##y_test

In [ ]:

NUM_FEATURES = x_train.shape[1]
print(NUM_FEATURES)


# Data Visualization 

## Histogram

In [ ]:
sb.histplot(data=features_ds_raw)

# Boxplot

In [ ]:
sb.boxplot(data=features_ds_raw)

## Correlation Matrix

In [ ]:

C_mat = features_ds_raw.corr()
C_mat


In [ ]:

fig = plt.figure(figsize = (15,15))
sb.heatmap(C_mat, vmax=.8, square=True)
plt.show()


## Filter Feature Selection Techniques

### Univariate Feature Selection

In [ ]:

f_val, p_val = f_regression(features_ds, np.array(labels_ds).ravel())


In [ ]:
f_val

In [ ]:
p_val

In [ ]:

feature_dictonary={'features':features_ds_raw.columns.tolist(), 'f_scores':f_val.tolist()}
features_f_score= pd.DataFrame(feature_dictonary).sort_values(by='f_scores', ascending=False).reset_index(drop=True)


In [ ]:
features_f_score

### Variance Threshold

In [ ]:

variance_filter = VarianceThreshold(threshold=0.25)
features_variance_selected = variance_filter.fit_transform(features_ds_raw)


In [ ]:
pd.DataFrame(features_variance_selected)

## Principal Component Analysis

## K-Means Clustering

## Linear Regression Model

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
model = LinearRegression()

In [ ]:
model.fit(x_train, y_train)

In [ ]:
y_pred = model.predict(x_test)

In [ ]:
mean_abs_error = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

In [ ]:
print("Mean Absolute Error:", mean_abs_error)
print("R-squared:", r2)

## DNN Architecture

In [ ]:
input_layer = tf.keras.layers.Input(shape=(NUM_FEATURES))

x = tf.keras.layers.Dense(units=1024, activation='relu', kernel_regularizer=regularizers.l2(0.01))(input_layer)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Dense(units=512, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Dense(units=256, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = tf.keras.layers.Dropout(0.25)(x)
x = tf.keras.layers.Dense(units=128, activation='relu', kernel_regularizer=regularizers.l2(0.01))(x)
x = tf.keras.layers.Dropout(0.25)(x)

output_layer = tf.keras.layers.Dense(units=1, activation='linear')(x)

Model = tf.keras.Model(input_layer, output_layer)


In [ ]:
Model.summary()

## Training Process

In [ ]:

callback = tf.keras.callbacks.EarlyStopping (
    monitor = 'mean_absolute_error',
    patience = 5,
    restore_best_weights = True
)


In [ ]:

Model.compile(loss='mean_absolute_error', optimizer=Adam(learning_rate=0.005), metrics=['mean_absolute_error'])


Model.load_weights("Model-02-004.h5")

In [ ]:

history = Model.fit (
    x_train, 
    y_train,
    validation_data = (x_test, y_test),
    epochs = 50,
    callbacks = [callback]
)


In [ ]:
Model.evaluate(x_test, y_test)

Model.save("FINAL-PROJECT-TRXX-TEXX.h5")

# Result Interpretation & Evaluation 

In [ ]:

plt.plot(history.history['mean_absolute_error']) 
plt.plot(history.history['val_mean_absolute_error']) 
plt.legend(['Training Set', 'Testing Set'], loc='upper left')
plt.xlabel('Epoch')
plt.ylabel('MAE')
plt.title('Model MAE')
plt.show()


In [ ]:
preds = Model.predict(x_test)

## Correlation Coefficient

In [ ]:
r2 = r2_score(y_test, preds)

In [ ]:
r2